# Install

In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

# Import

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Keypoint using MP Holistic

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             #mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
   # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh, rh])

# Setup Folders for Collection

In [73]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('M:\Work\MIT\MIT_Sign_Language_Recognition\AI\Data') 

# Actions that we try to detect
actions = np.array(['miss_new'])

# Thirty videos worth of data
start = 40
stop = 50
# Videos are going to be 30 frames in length
sequence_length = 30

In [74]:
for action in actions: 
    for sequence in range(start,stop):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [75]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start,stop):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(3000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# Preprocess Data and Create Labels and Features

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
actions = np.array(['angry','cry','disklike','easy',
                    'fine','forgot','full','happy','hard','hello',
                    'like','love','me','miss','sad','scare','stand','you'])
no_sequences = 30
sequence_length = 50
DATA_PATH = os.path.join('M:\Work\MIT\MIT_Sign_Language_Recognition\AI\DataTrain') 

In [4]:
label_map = {label:num for num, label in enumerate(actions)}

In [5]:
label_map

{'angry': 0,
 'cry': 1,
 'disklike': 2,
 'easy': 3,
 'fine': 4,
 'forgot': 5,
 'full': 6,
 'happy': 7,
 'hard': 8,
 'hello': 9,
 'like': 10,
 'love': 11,
 'me': 12,
 'miss': 13,
 'sad': 14,
 'scare': 15,
 'stand': 16,
 'you': 17}

In [6]:
sequences, labels = [], []
for action in actions:
    for sequence in range(sequence_length):
        window = []
        for frame_num in range(no_sequences):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [7]:
np.array(sequences).shape

(900, 30, 258)

In [8]:
np.array(labels).shape

(900,)

In [9]:
X = np.array(sequences)

In [10]:
X.shape

(900, 30, 258)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [14]:
y_test.shape

(45, 18)

# Build and Train LSTM Neural Network

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.layers import Dropout

In [16]:
og_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir="M:\Work\MIT\logs")

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])

Epoch 1/300
27/27 [==============================] - 3s 28ms/step - loss: 2.8689 - categorical_accuracy: 0.0795
Epoch 2/300
27/27 [==============================] - 1s 27ms/step - loss: 2.8717 - categorical_accuracy: 0.0725
Epoch 3/300
27/27 [==============================] - 1s 28ms/step - loss: 2.7163 - categorical_accuracy: 0.1135
Epoch 4/300
27/27 [==============================] - 1s 28ms/step - loss: 2.7686 - categorical_accuracy: 0.1205
Epoch 5/300
27/27 [==============================] - 1s 29ms/step - loss: 2.4741 - categorical_accuracy: 0.1719
Epoch 6/300
27/27 [==============================] - 1s 30ms/step - loss: 2.2402 - categorical_accuracy: 0.2000
Epoch 7/300
27/27 [==============================] - 1s 32ms/step - loss: 1.9576 - categorical_accuracy: 0.2667
Epoch 8/300
27/27 [==============================] - 1s 32ms/step - loss: 1.7188 - categorical_accuracy: 0.3392
Epoch 9/300
27/27 [==============================] - 1s 31ms/step - loss: 1.7458 - categorical_accuracy:

27/27 [==============================] - 1s 35ms/step - loss: 0.0017 - categorical_accuracy: 1.0000
Epoch 146/300
27/27 [==============================] - 1s 35ms/step - loss: 0.0014 - categorical_accuracy: 1.0000
Epoch 147/300
27/27 [==============================] - 1s 35ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 148/300
27/27 [==============================] - 1s 35ms/step - loss: 9.3734e-04 - categorical_accuracy: 1.0000
Epoch 149/300
27/27 [==============================] - 1s 34ms/step - loss: 8.3851e-04 - categorical_accuracy: 1.0000
Epoch 150/300
27/27 [==============================] - 1s 34ms/step - loss: 7.4455e-04 - categorical_accuracy: 1.0000
Epoch 151/300
27/27 [==============================] - 1s 34ms/step - loss: 6.7426e-04 - categorical_accuracy: 1.0000
Epoch 152/300
27/27 [==============================] - 1s 35ms/step - loss: 6.0592e-04 - categorical_accuracy: 1.0000
Epoch 153/300
27/27 [==============================] - 1s 35ms/step - loss: 5.4844

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 237,284
Trainable params: 237,284
Non-trai

In [19]:
res = model.predict(X_test)

1/1 [==============================] - 0s 302ms/step


In [20]:
actions[np.argmax(res[4])]

'2'

In [21]:
actions[np.argmax(y_test[4])]

'2'

In [22]:
model.save('action.h5')